In [21]:
from scipy.optimize import curve_fit
import numpy as np
import pandas as pd

df = pd.read_csv("covid_data.csv")
italy_data = df[df["location"]=="Italy"]
italy_data = italy_data.sort_values("date", ascending = True)



# Filtering the data for Italy between 2020-02-28 and 2020-03-20
italy_specific_period = italy_data.copy()
italy_specific_period = italy_specific_period[(italy_specific_period['date'] >= '2020-02-28') & (italy_specific_period['date'] <= '2020-03-20')]

italy_specific_period["total_cases"] = italy_specific_period["new_cases"].cumsum()


# Extracting the total cases and the corresponding dates
dates = italy_specific_period['date']
total_cases = italy_specific_period['total_cases']

# Defining the exponential function to fit
def exponential_func(x, a, b):
    return a * np.exp(b * x)

# Converting dates to a numerical format (days since the start date)
days_since_start = (pd.to_datetime(dates) - pd.to_datetime('2020-02-28')).dt.days

# Fitting the exponential function to the data
params, _ = curve_fit(exponential_func, days_since_start, total_cases, method="lm")

# Predicting total cases on 2020-03-20 using the fitted exponential function
day_to_predict = (pd.to_datetime('2020-03-20') - pd.to_datetime('2020-02-28')).days+1
predicted_cases = exponential_func(day_to_predict, *params)

# Actual total cases on 2020-03-20
actual_cases = italy_specific_period[italy_specific_period['date'] == '2020-03-20']['total_cases'].values[0]

# Calculating the difference
difference = actual_cases - predicted_cases
predicted_cases, actual_cases, difference


(49689.77953141087, 40635, -9054.779531410873)

In [20]:
day_to_predict

21

In [22]:
new_covid_data = pd.read_csv(f"covid_data.csv")

# Getting the latest data for each country to represent the current state
latest_new_data = new_covid_data.groupby('location').last()

In [1]:
# Feature description expense

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize

# Example dataset: Replace this with your actual dataset
data = {
    'TransactionID': [1, 2, 3, 4, 5],
    'Description': [
        "Coffee shop",
        "Supermarket grocery shopping",
        "Online subscription service",
        "Utility bill payment",
        "Coffee with friends"
    ]
}

df = pd.DataFrame(data)

# Step 1: Preprocess and vectorize transaction descriptions
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['Description'])

# Normalize the feature vectors to ensure better convergence
X_normalized = normalize(X, norm='l2')

# Step 2: Perform K-means clustering
# Note: Choose the number of clusters (n_clusters) based on your dataset and needs
n_clusters = 3
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(X_normalized)

# Step 3: Assign the cluster labels to each transaction
df['Cluster'] = kmeans.labels_

# Step 4: Review the results
print("Cluster centroids (TF-IDF vectors):")
print(kmeans.cluster_centers_)

print("\nTransactions with their assigned clusters:")
print(df[['TransactionID', 'Description', 'Cluster']])

# Optional: Analyze the clusters to label them manually
# This could involve reviewing the terms closest to each cluster's centroid
feature_names = vectorizer.get_feature_names_out()
for i in range(n_clusters):
    print(f"\nTop terms in cluster {i}:")
    centroid = kmeans.cluster_centers_[i]
    sorted_centroids = centroid.argsort()[::-1]
    for j in sorted_centroids[:10]:  # Adjust the number of terms you want to see
        print(f" {feature_names[j]} (score: {centroid[j]})")
